In [10]:
import cv2
import glob
import numpy as np
from  matplotlib import pyplot as plt
from collections import defaultdict 

filenames = glob.glob("sequence/*.jpg")
frames = [cv2.imread(img) for img in filenames]


frames= np.array(frames)

In [3]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import numpy as np

# import cv2
# import time


# class DetectorAPI:
#     def __init__(self, path_to_ckpt):
#         self.path_to_ckpt = path_to_ckpt

#         self.detection_graph = tf.Graph()
#         with self.detection_graph.as_default():
#             od_graph_def = tf.GraphDef()
#             with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
#                 serialized_graph = fid.read()
#                 od_graph_def.ParseFromString(serialized_graph)
#                 tf.import_graph_def(od_graph_def, name='')

#         self.default_graph = self.detection_graph.as_default()
#         self.sess = tf.Session(graph=self.detection_graph)

#         # Definite input and output Tensors for detection_graph
#         self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
#         # Each box represents a part of the image where a particular object was detected.
#         self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
#         # Each score represent how level of confidence for each of the objects.
#         # Score is shown on the result image, together with the class label.
#         self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
#         self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
#         self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

#     def processFrame(self, image):
#         # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
#         image_np_expanded = np.expand_dims(image, axis=0)
#         # Actual detection.
#         start_time = time.time()
#         (boxes, scores, classes, num) = self.sess.run(
#             [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
#             feed_dict={self.image_tensor: image_np_expanded})
#         end_time = time.time()

#         print("Elapsed Time:", end_time-start_time)

#         im_height, im_width,_ = image.shape
#         boxes_list = [None for i in range(boxes.shape[1])]
#         for i in range(boxes.shape[1]):
#             boxes_list[i] = (int(boxes[0,i,0] * im_height),
#                         int(boxes[0,i,1]*im_width),
#                         int(boxes[0,i,2] * im_height),
#                         int(boxes[0,i,3]*im_width))

#         return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])

#     def close(self):
#         self.sess.close()
#         self.default_graph.close()

# if __name__ == "__main__":
#     model_path = 'G:\\UNSW\\2020 T1\\Project\\Group_Component\\Group_Component\\faster_rcnn_inception_v2_coco_2018_01_28\\frozen_inference_graph.pb'
#     odapi = DetectorAPI(path_to_ckpt=model_path)
#     threshold = 0.7
    

# #     while True:
# #         r, img = cap.read()
# #         img = cv2.resize(img, (1280, 720))
#     frame_array = []
#     for i in frames:
#         img =i


#         # Visualization of the results of a detection.

     
#         boxes, scores, classes, num = odapi.processFrame(img)

#         # Visualization of the results of a detection.

#         for i in range(len(boxes)):
#             # Class 1 represents human
#             if classes[i] == 1 and scores[i] > threshold:
#                 box = boxes[i]
#                 cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
#         frame_array.append(img)

#         cv2.imshow("preview", img)
#         if cv2.waitKey(33) == 27:
#             break
# cv2.destroyAllWindows()



In [11]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared
    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1
    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]
    def update(self, rects):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            # return early as there are no centroids or tracking info
            # to update
            return self.objects
        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        # loop over the bounding box rectangles
#         print(enumerate(rects))
        
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
                        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())
            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value is at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()
            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]
            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()
            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue
                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)
                # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
                        # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        # return the set of trackable objects
        return self.objects

In [12]:

class People_In_Box(object):
    # Class to detect people in a box
    def __init__(self, frame, centers):
        self.people_in_box = 0
        self.frame = frame
        self.centers = centers

    # Function to count the people in the box
    def count_people(self, x1, y1, x2, y2):
        people_in_box = 0

        # Extracting the coordinates from the centroids
        for x, y in self.centers:
            cX = x
            cY = y
            # Increasing count of people inside the box
            if cX > x1 and cX < x2 and cY > y1 and cY < y2:
                people_in_box += 1

        # Returning the count
        return people_in_box
    # Function to count the people in the box
    def count_people_in_group(self, threshold):
        """
        Function for Task 3 to fix
        """
        # Establish box with threshold to include other nearby people
        people_in_group = 0
        people_alone = 0

        # Extracting the coordinates from the centroids
        for x, y in self.centers:
            x1 = x - threshold
            y1 = y - threshold
            x2 = x + threshold
            y2 = y + threshold

            # Calculating people in group within threshold
            people_in_group = self.count_people(x1, y1, x2, y2)
            # Differentiating single pedestrian from groups of people
            if people_in_group == 1:
                people_in_group = 0
                people_alone += 1
            # Draws the box if a group (more than 1 people) is detected
            elif people_in_group > 1:
                # Attempt to keep count value consistent throught but unsure if it is working
                people_in_group = self.count_people(x1, y1, x2, y2)
                # draw a rectangle
                cv2.rectangle(
                    self.frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

        # Returning the count of people witin a group as well as the frame with the drawn box
        return people_in_group, people_alone, self.frame


In [32]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

import cv2
import time


class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt

        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)

        # Definite input and output Tensors for detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')

    def processFrame(self, image):
        # Expand dimensions since the trained_model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Actual detection.
        start_time = time.time()
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
        end_time = time.time()

        #print("Elapsed Time:", end_time-start_time)

        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))

        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])
    
    def drawTrail(self, pts_dict, image):
        for img_ids in pts_dict.keys():
            pts = pts_dict[img_ids]
            for i in range(1,len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    continue
                cv2.line(image, pts[i - 1], pts[i], (0, 0, 255), 2)
        return image
    def close(self):
        self.sess.close()
        self.default_graph.close()



In [34]:

if __name__ == "__main__":
    model_path = 'C:\\Users\\abhij\\Desktop\\LectureNotes\\T1 2020\\COMP9517\\Group project\\comp9517\\faster_rcnn_inception_v2_coco_2018_01_28\\frozen_inference_graph.pb'
#     model_path = 'G:\\UNSW\\2020 T1\\Project\\Group_Component\\Group_Component\\ssd_mobilenet_v1_coco_2018_01_28\\frozen_inference_graph.pb'
    
    
    counter = 0
    
    odapi = DetectorAPI(path_to_ckpt=model_path)
    threshold = 0.8
    
    (H, W) = (None, None)
    boxd = []
    scored = []
    classed = []
    nums = []
#     while True:
#         r, img = cap.read()
#         img = cv2.resize(img, (1280, 720))
    frame_array = []
    ct = CentroidTracker()
    #Default dictionary of format {id:[[centroid[0],centroid[1]],..], id[...]}
    pts_dict = defaultdict(list)
    
    for i in frames:
        if W is None or H is None:
            (H, W) = frames.shape[:2]
        img =i
        # Visualization of the results of a detection.       
        boxes, scores, classes, num = odapi.processFrame(img)
#         boxd.append(boxes)
#         scored.append(scores)
#         classed.append(classes)
#         nums.append(num)
        # Visualization of the results of a detection.
#         boxd = []
        rects = []
        for i in range(len(boxes)):
            # Class 1 represents human
            if classes[i] == 1  and scores[i] > threshold:
                box = boxes[i]
                cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,255),2)
                rects.append(box)
        objects = ct.update(rects)    
        centers=[]
    # loop over the tracked objects
        for (objectID, centroid) in objects.items():
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            cv2.putText(img, text, (centroid[1] - 10, centroid[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.circle(img, (centroid[1], centroid[0]), 4, (0, 255, 0), 4)
            centers.append([centroid[1],centroid[0]])            
            pts_dict[objectID].append((centroid[1], centroid[0]))
            img = odapi.drawTrail(pts_dict , img)  
            
#         cv2.putText(img,text)
        people_inside = People_In_Box(img, centers)
        peopleInGroup, peopleAlone, img = people_inside.count_people_in_group(35)

                # Return the count of the people inside the drawn box
#         peopleInBox = people_inside.count_people(
#                     x_coord, y_coord, x_coord + width, y_coord + height)
                
        frame_array.append(img)

        cv2.imshow("preview", img)
        if cv2.waitKey(33) == 27:
            break
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
class groups(object):
    def __init__(self,coords,centers):
        self.coords = coords
        self.centers  = centers
    def people_in_groups(self,threshold,frame):
        for i in range(len(centers)-1):
            for k in range(i,len(centers)):
                if abs(centers[i][0]-center[i]

In [6]:
height, width, layers = frames[0].shape
size = (width,height)
out = cv2.VideoWriter('tensor_video_tracking_group.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()